In [1]:
!pip install pydub librosa

In [10]:
import numpy as np
import scipy.io.wavfile as wav
import glob
import os

class VoicemailComplianceSystem:
    def __init__(self, chunk_size_ms=100):
        self.chunk_size_ms = chunk_size_ms
        self.BEEP_FREQ = 1000
        self.FREQ_TOLERANCE = 50
        self.SILENCE_THRESHOLD = 0.01
        self.REQUIRED_SILENCE_MS = 1200

        self.COMPLIANCE_MESSAGE = (
            "Hi, this is ClearPath Finance calling regarding your account. "
            "Please call us back at 800-555-0199. Thank you."
        )

    def detect_beep(self, chunk, sample_rate):
        if len(chunk) < 2: return False
        fft_data = np.abs(np.fft.rfft(chunk * np.hamming(len(chunk))))
        freqs = np.fft.rfftfreq(len(chunk), d=1/sample_rate)
        peak_freq = freqs[np.argmax(fft_data)]
        peak_vol = np.max(fft_data)
        return (self.BEEP_FREQ - self.FREQ_TOLERANCE < peak_freq < self.BEEP_FREQ + self.FREQ_TOLERANCE) and peak_vol > 5

    def process_file(self, file_path):
        sample_rate, data = wav.read(file_path)
        if data.ndim > 1: data = data.mean(axis=1) # Mono conversion
        if data.dtype == np.int16: data = data / 32768.0

        chunk_samples = int(sample_rate * (self.chunk_size_ms / 1000))
        silence_duration_ms = 0

        for i in range(0, len(data), chunk_samples):
            current_time_ms = (i / sample_rate) * 1000
            chunk = data[i : i + chunk_samples]

            # 1. BEEP TRIGGER
            if self.detect_beep(chunk, sample_rate):
                return round((current_time_ms + 400) / 1000, 2), "Beep Detection"

            # 2. SILENCE TRIGGER
            rms_vol = np.sqrt(np.mean(chunk**2))
            if rms_vol < self.SILENCE_THRESHOLD:
                silence_duration_ms += self.chunk_size_ms
            else:
                silence_duration_ms = 0

            if silence_duration_ms >= self.REQUIRED_SILENCE_MS:
                return round(current_time_ms / 1000, 2), "Silence Detection"

        return None, "Manual Review"

# --- EXECUTION AND OUTPUT ---
def run_final_submission():
    folder_path = '/content/'
    wav_files = sorted(glob.glob(os.path.join(folder_path, "*.wav")))
    system = VoicemailComplianceSystem()

    print("="*80)
    print("CLEARPATH FINANCE - VOICEMAIL DROP AUDIT REPORT")
    print("="*80)
    print(f"{'FILE NAME':<25} | {'START TIME':<12} | {'TRIGGER METHOD'}")
    print("-" * 80)

    for path in wav_files:
        fname = os.path.basename(path)
        timestamp, method = system.process_file(path)

        if timestamp:
            print(f"{fname:<25} | {str(timestamp)+'s':<12} | {method}")
            print(f"  > ACTION: Playing Message: \"{system.COMPLIANCE_MESSAGE}\"\n")
        else:
            print(f"{fname:<25} | {'FAILED':<12} | {method}\n")

run_final_submission()

CLEARPATH FINANCE - VOICEMAIL DROP AUDIT REPORT
FILE NAME                 | START TIME   | TRIGGER METHOD
--------------------------------------------------------------------------------
vm1_output.wav            | 13.1s        | Silence Detection
  > ACTION: Playing Message: "Hi, this is ClearPath Finance calling regarding your account. Please call us back at 800-555-0199. Thank you."

vm2_output.wav            | 4.6s         | Beep Detection
  > ACTION: Playing Message: "Hi, this is ClearPath Finance calling regarding your account. Please call us back at 800-555-0199. Thank you."

vm3_output.wav            | 1.5s         | Beep Detection
  > ACTION: Playing Message: "Hi, this is ClearPath Finance calling regarding your account. Please call us back at 800-555-0199. Thank you."

vm4_output.wav            | 6.8s         | Silence Detection
  > ACTION: Playing Message: "Hi, this is ClearPath Finance calling regarding your account. Please call us back at 800-555-0199. Thank you."

vm5_out

In [11]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [13]:
from IPython.display import Audio, display
from pydub import AudioSegment
import os

def generate_demo_audio(input_file, message_file, trigger_time_s):
    # 1. Load the customer's greeting and your compliance message
    greeting = AudioSegment.from_file(input_file)
    compliance_msg = AudioSegment.from_file(message_file)

    # 2. Cut the greeting at the detected trigger point
    # trigger_time_s is in seconds, pydub works in milliseconds
    cut_point = int(trigger_time_s * 1000)
    trimmed_greeting = greeting[:cut_point]

    # 3. Combine them (Customer ends -> Message starts)
    final_call = trimmed_greeting + compliance_msg

    # 4. Export to a temporary file
    final_call.export("final_demo_output.mp3", format="mp3")

    print(f"Demo generated for {os.path.basename(input_file)}")
    print(f"The message drops at {trigger_time_s}s. Listen below:")

    # 5. Create the playable audio widget
    display(Audio("final_demo_output.mp3"))

In [17]:
generate_demo_audio('vm1_output.wav', 'clearpath_message.mp3.mp3', 13.1)

Demo generated for vm1_output.wav
The message drops at 13.1s. Listen below:
